In [1]:
pip install tangelo-gc

  Using cached tangelo_gc-0.4.2-py3-none-any.whl.metadata (912 bytes)
  Using cached h5py-3.11.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.5 kB)
  Using cached bitarray-2.9.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (34 kB)
  Using cached openfermion-1.6.1-py3-none-any.whl.metadata (10 kB)
  Using cached deprecation-2.1.0-py2.py3-none-any.whl.metadata (4.6 kB)
  Using cached PubChemPy-1.0.4-py3-none-any.whl
Using cached tangelo_gc-0.4.2-py3-none-any.whl (598 kB)
Using cached bitarray-2.9.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (296 kB)
Using cached h5py-3.11.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.4 MB)
Using cached openfermion-1.6.1-py3-none-any.whl (1.2 MB)
Using cached deprecation-2.1.0-py2.py3-none-any.whl (11 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
try:
    import os
except ImportError:
    print ("os error")
    
!pip install tangelo

  Using cached tangelo-0.10.0.tar.gz (2.5 MB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [8 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/tmp/pip-install-_jn1vx0f/tangelo_90dd5d5eba0440a28cd7c89088d590cd/setup.py", line 24, in <module>
          package_data={"tangelo": rcollect("tangelo/pkgdata")},
        File "/tmp/pip-install-_jn1vx0f/tangelo_90dd5d5eba0440a28cd7c89088d590cd/setup.py", line 8, in rcollect
          return sum(map(lambda x: map(lambda y: x[0] + "/" + y, x[2]), os.walk(path)), [])
      TypeError: can only concatenate list (not "map") to list
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generati

In [3]:
pip install PySCF

  Using cached pyscf-2.5.0-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.1 kB)
Using cached pyscf-2.5.0-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (47.3 MB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
from tangelo import SecondQuantizedMolecule as SQMol


# cooh = ?????
li2=  """Li 0.  0. 0.
         Li 3.0 0. 0. """

# 2 electrons in 2 orbitals
fo = [0,1]+[i for i in range(4,28)]

# Runs RHF calculation
mol_Li2 = SQMol(li2, q=0, spin=0, basis='6-31g(d,p)', frozen_orbitals=fo, symmetry=True)

# Runs ROHF calculation
mol_Li2_t = SQMol(li2, q=0, spin=2, basis="6-31g(d,p)", frozen_orbitals=fo, symmetry=True)

In [5]:
# Symmetry labels and occupations for frozen core and active orbitals
print("  #  Energy  Symm Occ")
for i in range(4):
    print(f"{i+1:3d}{mol_Li2.mo_energies[i]: 9.4f}  {mol_Li2.mo_symm_labels[i]}   {int(mol_Li2.mo_occ[i])}")

# Active electrons, Active orbitals
print(f"Number of active electrons: {mol_Li2.n_active_electrons}")
print(f"Number of active orbtials: {mol_Li2.n_active_mos}")

  #  Energy  Symm Occ
  1  -2.4479  A1g   2
  2  -2.4478  A1u   2
  3  -0.1716  A1g   2
  4   0.0129  A1u   0
Number of active electrons: 2
Number of active orbtials: 2


###Understanding the VQE Solver 

In [6]:
from tangelo.algorithms.variational import VQESolver, BuiltInAnsatze
from tangelo.algorithms.classical import FCISolver

# Dictionary of resources for each algorithm
algorithm_resources = dict()

# Ground state energy calculation with VQE, reference values with FCI
vqe_options = {"molecule": mol_Li2, "ansatz": BuiltInAnsatze.UCCSD}
vqe_solver = VQESolver(vqe_options)
vqe_solver.build()
vqe_energy = vqe_solver.simulate()
print("\n Ground Singlet state")
print(f"VQE energy = {vqe_energy}")
print(f"CASCI energy = {FCISolver(mol_Li2).simulate()}")
algorithm_resources["vqe_ground_state"] = vqe_solver.get_resources()

# First excited state energy calculation with VQE, reference values with FCI
vqe_options = {"molecule": mol_Li2_t, "ansatz": BuiltInAnsatze.UpCCGSD}
vqe_solver_t = VQESolver(vqe_options)
vqe_solver_t.build()
vqe_energy_t = vqe_solver_t.simulate()
print("\n Lowest Triplet state")
print(f"VQE energy = {vqe_energy_t}")
print(f"CASCI energy = {FCISolver(mol_Li2_t).simulate()}")
algorithm_resources["vqe_triplet_state"] = vqe_solver_t.get_resources()

#stor ground state fo rlater
ground_vqe = vqe_energy


 Ground Singlet state
VQE energy = -14.869620361804408
CASCI energy = -14.869620362826856

 Lowest Triplet state
VQE energy = -14.853462489027226
CASCI energy = -14.853462489027095


In [7]:
energy_diff = (vqe_energy_t-vqe_energy)
# convert to joules
energy_diff = -energy_diff*(-2.18*10**-18)
print("Energy difference is:", energy_diff)
h = 6.626*10**-34
v = energy_diff/h
print("frequency is:", v)
c = 299792458
w = c/v
print("wavelength is:", w)

Energy difference is: 3.522416265425641e-20
frequency is: 53160523172738.32
wavelength is: 5.639381257138173e-06


In [8]:
#Getting next excited states
# Add initial VQE optimal circuit to the deflation circuits list
deflation_circuits = [vqe_solver.optimal_circuit.copy()]

# Calculate first and second excited states by adding optimal circuits to deflation_circuits
dict_vqe = {}
for i in range(3):
    vqe_options = {"molecule": mol_Li2, "ansatz": BuiltInAnsatze.UpCCGSD, 
                   "deflation_circuits": deflation_circuits, "deflation_coeff": 0.4}
    vqe_solver = VQESolver(vqe_options)
    vqe_solver.build()
    vqe_energy = vqe_solver.simulate()
    print(f"Excited state #{i+1} \t VQE energy = {vqe_energy}")
    algorithm_resources[f"vqe_deflation_state_{i+1}"] = vqe_solver.get_resources()
    dict_vqe[i+1] =  vqe_energy
    deflation_circuits.append(vqe_solver.optimal_circuit.copy())

Excited state #1 	 VQE energy = -14.784036828633347
Excited state #2 	 VQE energy = -14.6801963328278
Excited state #3 	 VQE energy = -14.469625901056558


In [9]:
# initiailizing dictionaries
dict_diff_vqe = {}
vqe_frequencies = {}
vqe_wavelengths = {}

# for each level, difference between states, appending to list appending to dictionary
# each corresponding label represents the nth energy state.
for i in range(1,len(dict_vqe)+1):
    list_diff_energy= []
    for j in range(1,i):
        list_diff_energy.append((dict_vqe[i] - dict_vqe[j])*(-2.18*10**-18))
    # remember that ground_vqe holds the ground state.
    list_diff_energy.append(dict_vqe[i]*(-2.18*10**-18) - ground_vqe*(-2.18*10**-18))
    dict_diff_vqe[i] = list_diff_energy
print("Our different energy differences (J) are:", dict_diff_vqe) #converting to J


# for key, value in dict_diff_vqe.items():
#     vqe_frequencies[key] = [freq * 5 for freq in value]
# print(vqe_frequencies) 

def find_v_and_w(value):
    h = 6.626*10**-34
    v = energy_diff/h
    c = 299792458
    w = c/v
    return v, w

for key, value in dict_diff_vqe.items():
    for freq in value:
        vqe_frequencies.setdefault(key, []).append(find_v_and_w(freq)[0])
        vqe_wavelengths.setdefault(key, []).append(find_v_and_w(freq)[1])
    
print("Our corresponding frequencies (Hz) are:", vqe_frequencies)
print("Our corresponding wavelengths (m) are:", vqe_wavelengths)

Our different energy differences (J) are: {1: [-1.865721023129175e-19], 2: [-2.2637228085609394e-19, -4.129443831690091e-19], 3: [-6.854158221174006e-19, -4.590435412613067e-19, -8.71987924430317e-19]}
Our corresponding frequencies (Hz) are: {1: [53160523172738.32], 2: [53160523172738.32, 53160523172738.32], 3: [53160523172738.32, 53160523172738.32, 53160523172738.32]}
Our corresponding wavelengths (m) are: {1: [5.639381257138173e-06], 2: [5.639381257138173e-06, 5.639381257138173e-06], 3: [5.639381257138173e-06, 5.639381257138173e-06, 5.639381257138173e-06]}


In [10]:
# Let's try to "freeze" electron orbitals for our new molecules

In [11]:
# cooh = ?????

# diatomic oxygen
o2_xyz = [("O", (0., 0., 0.)), ("O", (0., 0., 1.21))]

##########################

from tangelo import SecondQuantizedMolecule
o2_6311gdp = SecondQuantizedMolecule(o2_xyz, q=0, spin=2)
print(f"{o2_6311gdp.n_active_mos} active molecular orbitals")
print(f"{o2_6311gdp.n_active_electrons} active electrons")

##########################

basis_sets = [
    "STO-3G",       # Simple zeta, minimal basis.
    "3-21G",        # Double zeta.
    "6-31G",        # Double zeta with more Gaussian primitives.
    "6-31G(d,p)",   # Polarization functions (+ 5 d-orbitals for all atoms except H, +3 p-orbitals for H atoms) added.
    "6-311G(d,p)",  # Triple zeta with polarization functions.
    "6-311+G(d,p)", # Triple zeta with polarization functions and diffuse functions.
    "cc-pvqz",      # Quadruple zeta.
    "cc-pv5z"       # Quintuple zeta.
]

###
o2_6311gdp.mo_occ
###
from tangelo.toolboxes.molecular_computation.frozen_orbitals import get_orbitals_excluding_homo_lumo

# Selecting HOMO-3 to LUMO+3 orbitals.
frozen = get_orbitals_excluding_homo_lumo(o2_6311gdp, homo_minus_n=6, lumo_plus_n=5)

o2_6311gdp_frozen = SecondQuantizedMolecule(o2_xyz, q=0, spin=2,
                                            basis="6-311G(d,p)",
                                            frozen_orbitals=frozen)
print(f"{o2_6311gdp_frozen.n_active_mos} active molecular orbitals")
print(f"{o2_6311gdp_frozen.n_active_electrons} active electrons")
###

##########################

# o2_6311gdp = SecondQuantizedMolecule(o2_xyz, q=0, spin=2,
#                                      basis="6-311G(d,p)", 
#                                      frozen_orbitals=None)
# print(f"{o2_6311gdp.n_active_mos} active molecular orbitals")
# print(f"{o2_6311gdp.n_active_electrons} active electrons")

# # Runs RHF calculation
# mol_O2 = SQMol(o2, q=0, spin=0, basis='6-31g(d,p)', frozen_orbitals=fo, symmetry=True)

# # Runs ROHF calculation
# mol_O2_t = SQMol(o2, q=0, spin=2, basis="6-31g(d,p)", frozen_orbitals=fo, symmetry=True)

8 active molecular orbitals
12 active electrons
34 active molecular orbitals
12 active electrons
